# 🤖 XGBoost model NEW DATA 2

**best_params = {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 7, 'min_data_in_leaf': 10, 'n_estimators': 100, 'subsample': 0.8}**

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [2]:
data= pd.read_csv('../data/data_imputed/data_processed_newdata2.csv')

In [3]:
'''
Seleccionamos todas las filas de 'data' donde el valor de la columna 'x_e_out [-]' no es nulo, y lo almacenamos en un nuevo dataframe 'y_no_null'.
'''
no_null = data[data['x_e_out [-]'].notnull()]
no_null.reset_index(drop=True, inplace=True)
no_null= no_null.drop('id', axis=1)
no_null


,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],author_Inasaka,author_Janssen,author_Kossolapov,author_Mortimore,author_Peskov,author_Richenderfer,author_Thompson,author_Weatherhead,author_Williams,geometry_plate,geometry_tube
0,7.000,3770.0,0.1754,9.417212,16.167721,432.0,3.6,0,0,0,0,0,0,1,0,0,0,1
1,7.718,6049.0,-0.0416,10.300000,10.300000,762.0,6.2,0,0,0,0,0,0,1,0,0,0,1
2,13.790,2034.0,0.0335,7.700000,7.700000,457.0,2.5,0,0,0,0,0,0,1,0,0,0,1
3,13.790,3679.0,-0.0279,5.600000,15.200000,2134.0,3.0,0,0,0,0,0,0,0,0,0,0,0
4,17.240,3648.0,-0.0711,9.417212,16.167721,696.0,3.6,0,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21224,12.070,3244.2,-0.0195,9.417212,16.167721,152.0,5.4,0,0,0,0,0,0,1,0,0,0,1
21225,15.512,3648.0,-0.0487,4.700000,4.700000,318.0,9.0,0,0,0,0,0,0,1,0,0,0,1
21226,12.790,1736.0,0.0886,9.417212,16.167721,591.0,2.3,0,0,0,0,0,0,1,0,0,0,1
21227,18.270,658.0,-0.1224,3.000000,3.000000,150.0,2.3,0,0,0,0,0,0,1,0,0,0,1


In [4]:
'''
Seleccionamos todas las filas de 'data' donde el valor de la columna 'x_e_out [-]' es nulo, y lo almacenamos en un nuevo dataframe 'target_null'.
'''
null = data[data['x_e_out [-]'].isnull()]
null.reset_index(drop=True, inplace=True)
null

,id,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],author_Inasaka,author_Janssen,author_Kossolapov,author_Mortimore,author_Peskov,author_Richenderfer,author_Thompson,author_Weatherhead,author_Williams,geometry_plate,geometry_tube
0,4,13.790,686.0,NaN,11.100000,11.100000,457.0,2.8,0,0,0,0,0,0,1,0,0,0,1
1,7,18.000,750.0,NaN,10.000000,10.000000,1650.0,2.2,0,0,0,0,1,0,0,0,0,0,1
2,10,12.776,2349.0,NaN,1.900000,1.900000,152.0,3.2,0,0,0,0,0,0,1,0,0,0,1
3,12,6.890,7500.0,NaN,9.417212,16.167721,1930.0,4.8,0,0,0,0,0,0,1,0,0,0,1
4,23,15.510,1355.0,NaN,5.600000,15.200000,2134.0,2.1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10410,31633,11.030,2904.6,NaN,11.500000,11.500000,1754.2,2.0,0,0,0,0,0,0,1,0,0,0,1
10411,31634,1.010,2000.0,NaN,15.000000,120.000000,10.0,6.2,0,0,0,0,0,1,0,0,0,1,0
10412,31637,13.790,688.0,NaN,9.417212,16.167721,457.0,2.3,0,0,0,0,0,0,0,1,0,0,1
10413,31640,13.790,1537.8,NaN,4.700000,4.700000,617.0,3.9,0,0,0,0,0,0,1,0,0,0,1


In [5]:
# Creamos una copia del DataFrame original
no_null = no_null.copy()

In [6]:
# Variables independientes
X = no_null.drop('x_e_out [-]', axis=1)
# Variable dependiente
y = no_null['x_e_out [-]']

import re 
# Preprocess column names
X.columns = [re.sub(r'[\[\]<>\s]+', '_', col) for col in X.columns]
X.columns

import re 
# Preprocess column names
y = pd.DataFrame(y)
y.columns = [re.sub(r'[\[\]<>\s]+', '_', col) for col in y.columns]
y.columns

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
y_train.shape, y_test.shape

((16983, 1), (4246, 1))

In [8]:
param_grid = {
    'learning_rate': [0.01, 0.1],
    'max_depth': [4, 7],
    'n_estimators': [100, 300],
    'subsample': [0.4, 0.8],
    'colsample_bytree': [0.2, 0.6],
    'min_data_in_leaf': [10, 50]
}

In [9]:
model = xgb.XGBRegressor()

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5)

In [10]:
import warnings
warnings.filterwarnings("ignore")

grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

[19:16:57] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_data_in_leaf" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:16:57] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_data_in_leaf" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:16:57] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_data_in_leaf" } might not be 

In [11]:
print(best_params)
print(best_estimator)

{'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 7, 'min_data_in_leaf': 10, 'n_estimators': 100, 'subsample': 0.8}
XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.6,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
             min_data_in_leaf=10, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0, ...)


In [12]:
# Entrenamos el modelo con los mejores parámetros
y_pred = best_estimator.predict(X_test)

In [13]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)

Mean Squared Error (MSE): 0.005614614151029508
Root Mean Squared Error (RMSE): 0.07493072901706954
Mean Absolute Error (MAE): 0.05189016511130748
R-squared (R2): 0.4472495943258248


In [1]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Assuming importance_xgb contains the feature importances from the XGB model
# Assuming preds contains the predictions from the XGB model

# Scatter plot of actual values vs predicted values
plt.figure(figsize=(8, 6))  # Ajusta el tamaño de la figura según tus necesidades
plt.scatter(y_test, y_pred, color='blue', label='Actual', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', alpha=0.5, linewidth=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Scatter Plot: Predicted vs. Actual')
plt.legend(['Reference Line', 'Predicted vs. Actual'])

plt.tight_layout()
plt.show()

NameError: name 'y_test' is not defined

<Figure size 800x600 with 0 Axes>

### 📤 Submission

In [18]:
x_df_null = null.drop('x_e_out [-]', axis=1)
x_df_null = x_df_null.drop(columns=["id"])
y_df_null = null['x_e_out [-]']
# Realizar predicciones para los datos imputados
y_pred1 = best_estimator.predict(x_df_null)
# Predicciones en la columna objetivo (y)
null['x_e_out [-]'] = y_pred1
# Guardar los datos en un archivo para la competición de Kaggle
null.to_csv("../submissions/xgb_pred2_reg_newdata2.csv", index=False)


# Leer el archivo CSV
xgboost_new2 = pd.read_csv("../submissions/xgb_pred2_reg_newdata2.csv")
# Mantener solo las columnas "id" y "x_e_out [mm]"
columnas_deseadas = ["id", "x_e_out [-]"]
xgboost_new2 = xgboost_new2[columnas_deseadas]
# Guardar el DataFrame resultante en un nuevo archivo CSV
xgboost_new2.to_csv("../submissions/xgb_pred2_reg_newdata2_sub.csv", index=False)

In [19]:
# Leer el archivo CSV
xgboost_new2 = pd.read_csv("../submissions/xgb_pred2_reg_newdata2.csv")
# Mantener solo las columnas "id" y "x_e_out [mm]"
columnas_deseadas = ["id", "x_e_out [-]"]
xgboost_new2 = xgboost_new2[columnas_deseadas]
# Guardar el DataFrame resultante en un nuevo archivo CSV
xgboost_new2.to_csv("../submissions/xgb_pred2_reg_newdata2_sub.csv", index=False)